<a href="https://colab.research.google.com/github/mmartigani/RoadMapIA/blob/main/35_Correos_Maliciosas_GTP_GMAIL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""En este caso práctico, se propone al alumno el desarrollo de una aplicación que
permita detectar de manera automática si un correo electrónico que llega a su bandeja
de entrada es malicioso (SPAM o Phishing) o es benigno.
Para ello, el alumno debe implementar un programa en Python que acceda de manera
programática a los servicios de OpenAI y le permita obtener la clasificación
del correo electrónico utilizando un LLM.
La aplicación debe ser lo más automática posible, integrandose con el servicio
de correo electrónico del alumno para leer automáticamente los mensajes sin abrir
de su bandeja de entrada y proporcionar un veredicto antes de que se abran.

In [ ]:
#!pip install openai==0.28

In [ ]:
import openai
with open("/content/drive/MyDrive/api-keys/secret-key.txt") as f:
  openai.api_key = f.readline()

In [ ]:
def obtener_completion(prompt, model="gpt-3.5-turbo"):
  mensaje = [{"role": "user", "content": prompt}]
  respuesta = openai.ChatCompletion.create(
      model=model,
      messages=mensaje,
      temperature=0, # Este hiperparámetro controla la aleatoriedad del modelo
  )
  return respuesta.choices[0].message["content"]

In [ ]:
def clasificar_email(correo):
  """Esta función identifica si un correo electrónico es malicioso o benigno."""

  prompt = f"""Actúa como si fueses el mayor experto en Ciberseguridad y
  análisis de correos electrónicos del mundo. A continuación te voy
  a proporcionar un email delimitado por triples comillas simples.
  Tu tarea es determinar si ese correo electrónico es malicioso (SPAM o Phishing) o Benigno.
  Para ello, realiza las siguientes acciones:
1. Analiza muy en detalle todo el contenido del correo electrónico
2. Extrae los patrones que consideras sospechosos desde el punto de vista de la seguridad
3. Analiza estos patrones y razona detalladamente el motivo por el que pueden ser maliciosos en el contexto del mensaje completo
4. Determina si el email es malioso o benigno.

La salida que debes proporcionarme debe ser un JSON con las claves que se muestran a continuación. No debes proporcionar nada más como salida.
{{\"clasificacion\":<MALICIOSO o BENIGNO>}}

Email: '''{correo}'''
"""
  return obtener_completion(prompt)

In [ ]:
correo1 = """Hello Dear. I have a proposition for you, this however is not mandatory nor will I in any manner compel you to honor against your will.Let me start by introducing myself.
I am  Mr Author Magnus, manager crew team. I have a mutual beneficial business suggestion for you and a very Important
 matter that we both had to engage in.

1. Can you handle the project?

2. Can I give you this trust ?

3. Can we work together as partner?

Absolute confidentiality is required from you.
Besides I will use my connection to get some documents to back up the fund so that the fund can not be question by any authority.
More information await you in my next response to your email.

Treat as urgent.

Yours Sincerely

Author Magnus."""

In [ ]:
correo2 = """Hola Santiago:

Hay una nueva respuesta a "Kali Linux en Hyper-V" en tu curso "Curso completo de Hacking Ético y Ciberseguridad"

Utiliza el botón de abajo para ver la respuesta e indicar si ha sido útil. Allí encontrarás un enlace para "dejar de seguir" si prefieres no recibir notificaciones de respuestas futuras."""

In [ ]:
clasificar_email(correo2)

In [ ]:
"""Para integrar la aplicación con el servicio de correo electrónico de Gmail,
lo primero que debemos hacer es habilitar la API de Gmail y obtener
las credenciales necesarias para poder invocarla.
En este enlace podéis encontrar el procedimiento oficial:
 https://developers.google.com/gmail/api/quickstart/python?hl=es-419
En este enlace podéis encontrar la referencia de la API de Gmail:
 https://developers.google.com/gmail/api/reference/rest?hl=es-419

In [ ]:
#!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']


def main():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('/content/drive/MyDrive/api-keys/token.json'):
        creds = Credentials.from_authorized_user_file('/content/drive/MyDrive/api-keys/token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'C:\\Users\\santiago\\Downloads\\credenciales.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        # Call the Gmail API
        # service = build('gmail', 'v1', credentials=creds)
        # results = service.users().labels().list(userId='me').execute()
        # labels = results.get('labels', [])

        service = build('gmail', 'v1', credentials=creds)

        results = service.users().messages().list(userId='me', labelIds=['INBOX'], q='is:unread').execute()

        for msg in results['messages']:
          mensaje = service.users().messages().get(userId='me', id=msg['id']).execute()
          print(f"Analizando el correo: {mensaje['snippet']}")
          print(f"El correo electrónico con id {msg['id']} es: {clasificar_email(mensaje['snippet'])}\n")

    except HttpError as error:
        # TODO(developer) - Handle errors from gmail API.
        print(f'An error occurred: {error}')


if __name__ == '__main__':
    main()